# Training a Fast Whisper Tokenizer for Tibetan

This notebook demonstrates how to train a Whisper tokenizer for Tibetan text using the `PreTrainedTokenizerFast` class, which correctly supports the `train_new_from_iterator` method.

We'll follow these steps:
1. Load a fast tokenizer from the Whisper model
2. Prepare a Tibetan text corpus
3. Train the tokenizer on the Tibetan corpus
4. Test and evaluate the tokenizer
5. Save the trained tokenizer

In [30]:
# Install necessary packages
!pip install transformers tokenizers tqdm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import os
from pathlib import Path
import numpy as np
from transformers import PreTrainedTokenizerFast, WhisperProcessor
from tqdm.notebook import tqdm

## 1. Load the Base Fast Tokenizer

We'll load the tokenizer from the Whisper model using `PreTrainedTokenizerFast` which supports the `train_new_from_iterator` method.

In [8]:
# Define the base model
BASE_MODEL = "/home/gangagyatso/Desktop/stt-bpe-trainer/data/tokenizer_bpe/bpe_tokenizer.json"

# Load the fast tokenizer
print(f"🚀 Loading fast tokenizer from {BASE_MODEL}...")
tokenizer = PreTrainedTokenizerFast.from_pretrained(BASE_MODEL)
print("✅ Fast tokenizer loaded successfully")

# Check vocabulary size
print(f"Vocabulary size: {len(tokenizer)}")

🚀 Loading fast tokenizer from /home/gangagyatso/Desktop/stt-bpe-trainer/data/tokenizer_bpe/bpe_tokenizer.json...


ValueError: Calling PreTrainedTokenizerFast.from_pretrained() with the path to a single file or url is not supported for this tokenizer. Use a model identifier or the path to a directory instead.

In [46]:
# Define output directory
OUTPUT_DIR = "data/whisper_tokenizer"


# Create directory if it doesn't exist
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [47]:

# Save the tokenizer
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"💾 Trained tokenizer saved to: {OUTPUT_DIR}")

💾 Trained tokenizer saved to: data/whisper_tokenizer


In [48]:
# Test loading it back
loaded_tokenizer = PreTrainedTokenizerFast.from_pretrained(OUTPUT_DIR)
print(f"✅ Successfully loaded tokenizer with {len(loaded_tokenizer)} tokens")

✅ Successfully loaded tokenizer with 51865 tokens


## 2. Prepare the Tibetan Corpus

Create an iterator for the Tibetan corpus.

In [35]:
# Path to your Tibetan corpus file
CORPUS_FILE = "data/corpus/mergedcorpus.txt"

# Check if the file exists
if not os.path.exists(CORPUS_FILE):
    print(f"❌ Corpus file not found: {CORPUS_FILE}")
else:
    print(f"✅ Found corpus file: {CORPUS_FILE}")
    
    # Look at a sample of the corpus
    with open(CORPUS_FILE, "r", encoding="utf-8") as f:
        sample_lines = [next(f).strip() for _ in range(3) if f]
    
    print("\nSample of corpus:")
    for i, line in enumerate(sample_lines):
        print(f"Line {i+1}: {line[:100]}..." if len(line) > 100 else f"Line {i+1}: {line}")

✅ Found corpus file: data/corpus/mergedcorpus.txt

Sample of corpus:
Line 1: ཨེ། སུའི་སྣང་བར་དེ་ཡོད་ན་
Line 2: སེམས་ཅན་ཐམས་ཅད་ཀྱི་ཚབ་ལ་གཞན་བསམ་ཚར་དུས་བདག་འཛིན་ཡོད་མ་རེད་
Line 3: རང་ཐོག་ལ་དངོས་སུ་སྙིང་རྗེ་ཡིས་འབྲེལ་བ་ཡོང་གི་ཡོད་རེད་


In [36]:
# Define corpus iterator function
def corpus_iterator():
    with open(CORPUS_FILE, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:  # Skip empty lines
                yield line

## 3. Test Initial Tokenizer on Tibetan

Let's first see how the original tokenizer handles Tibetan text.

In [37]:
# Test encoding a sentence
sentence = "བདེ་ལེགས་"
output = tokenizer.encode(sentence)
print("Tokens:", output.tokens)
print("IDs:", output.ids)

AttributeError: 'list' object has no attribute 'tokens'

In [38]:
# The list of token IDs you want to decode
ids_to_decode = [50258, 50363, 156, 121, 244, 156, 121, 239, 156, 121, 118, 156, 120, 233, 156, 121, 96, 156, 121, 118, 156, 121, 224, 156, 121, 99, 156, 120, 233, 50257]

# Use the decode method to convert IDs back to text
decoded_text = loaded_tokenizer.decode(ids_to_decode)

print(f"IDs: {ids_to_decode}")
print(f"Decoded Text: {decoded_text}")
print("token len: ", len(ids_to_decode))


IDs: [50258, 50363, 156, 121, 244, 156, 121, 239, 156, 121, 118, 156, 120, 233, 156, 121, 96, 156, 121, 118, 156, 121, 224, 156, 121, 99, 156, 120, 233, 50257]
Decoded Text: <|startoftranscript|><|notimestamps|>བདེ་ལེགས་<|endoftext|>
token len:  30


In [39]:
# Example Tibetan text
tibetan_example = "བྱང་ཆུབ་ཀྱི་སེམས་རྣམ་པ་གཉིས་ཡོད་རེད།"

# Tokenize with the original tokenizer
tokens = tokenizer.tokenize(tibetan_example)
token_ids = tokenizer.encode(tibetan_example)

print("Original tokenizer results:")
print(f"Text: {tibetan_example}")
print(f"Token count: {len(tokens)}")
print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids}")

# Check roundtrip
decoded = tokenizer.decode(token_ids)
print(f"\nRoundtrip: {tibetan_example in decoded}")
print(f"Decoded text: {decoded}")

Original tokenizer results:
Text: བྱང་ཆུབ་ཀྱི་སེམས་རྣམ་པ་གཉིས་ཡོད་རེད།
Token count: 108
Tokens: ['à', '½', 'ĸ', 'à', '¾', '±', 'à', '½', 'Ħ', 'à', '¼', 'ĭ', 'à', '½', 'Ĩ', 'à', '½', '´', 'à', '½', 'ĸ', 'à', '¼', 'ĭ', 'à', '½', 'Ģ', 'à', '¾', '±', 'à', '½', '²', 'à', '¼', 'ĭ', 'à', '½', '¦', 'à', '½', 'º', 'à', '½', 'ĺ', 'à', '½', '¦', 'à', '¼', 'ĭ', 'à', '½', '¢', 'à', '¾', '£', 'à', '½', 'ĺ', 'à', '¼', 'ĭ', 'à', '½', 'Ķ', 'à', '¼', 'ĭ', 'à', '½', 'Ĥ', 'à', '½', 'ī', 'à', '½', '²', 'à', '½', '¦', 'à', '¼', 'ĭ', 'à', '½', '¡', 'à', '½', '¼', 'à', '½', 'ĳ', 'à', '¼', 'ĭ', 'à', '½', '¢', 'à', '½', 'º', 'à', '½', 'ĳ', 'à', '¼', 'į']
Token IDs: [50258, 50363, 156, 121, 244, 156, 122, 109, 156, 121, 226, 156, 120, 233, 156, 121, 228, 156, 121, 112, 156, 121, 244, 156, 120, 233, 156, 121, 222, 156, 122, 109, 156, 121, 110, 156, 120, 233, 156, 121, 99, 156, 121, 118, 156, 121, 246, 156, 121, 99, 156, 120, 233, 156, 121, 95, 156, 122, 96, 156, 121, 246, 156, 120, 233, 156, 121, 242, 156, 120, 2

## 4. Train the Tokenizer on Tibetan Corpus

Now we'll train the tokenizer on our Tibetan corpus to improve its handling of Tibetan text.

In [40]:
# Set target vocabulary size
TARGET_VOCAB_SIZE = len(tokenizer) + 10000

print(f"🧠 Original vocabulary size: {len(tokenizer)}")
print(f"🎯 Target vocabulary size: {TARGET_VOCAB_SIZE}")

print("⏳ Starting tokenizer training... This may take a few minutes.")

# Train the tokenizer
new_tokenizer = tokenizer.train_new_from_iterator(corpus_iterator(), vocab_size=TARGET_VOCAB_SIZE, )

print("✅ Training complete!")
print(f"📈 New vocabulary size: {len(new_tokenizer)}")

🧠 Original vocabulary size: 51865
🎯 Target vocabulary size: 61865
⏳ Starting tokenizer training... This may take a few minutes.



✅ Training complete!
📈 New vocabulary size: 3059


## 5. Test the Trained Tokenizer

Let's check how the newly trained tokenizer handles Tibetan text.

In [41]:
# Test examples
test_examples = [
    "བྱང་ཆུབ་ཀྱི་སེམས་རྣམ་པ་གཉིས་ཡོད་རེད།",  # Simple sentence
    "བོད་ཀྱི་སྐད་ཡིག་ནི་ལོ་རྒྱུས་ཧ་ཅང་རིང་པོ་ཡོད་པའི་སྐད་ཡིག་ཅིག་རེད།",  # Another example
    "བྱང་ཆུབ་ཀྱི་སེམས་རྣམ་པ་གཉིས་ཡོད་རེད། བྱང་ཆུབ་ཀྱི་སེམས་ཡོད་ན་ཡེ་ཤེས་ལྷ་ལ་འགྱུར་འགྲོ་གི་ཡོད་རེད། བྱང་ཆུབ་ཀྱི་སེམས་མེད་ན། དེ་ནས་འདི་"
]

# Function to analyze tokenization
def analyze_tokenization(tokenizer, text, name="Tokenizer"):
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.encode(text)
    decoded = tokenizer.decode(token_ids)
    
    print(f"\n--- {name} Results ---")
    print(f"Text: {text}")
    print(f"Token count: {len(tokens)}")
    print(f"Tokens: {tokens}")
    print(f"Roundtrip successful: {text in decoded}")
    
    return len(tokens)

In [42]:
# Compare tokenization between original and new tokenizer
for i, example in enumerate(test_examples):
    print(f"\n==== Test Example {i+1} ====\n")
    
    # Test original tokenizer
    orig_count = analyze_tokenization(tokenizer, example, "Original Tokenizer")
    
    # Test new tokenizer
    new_count = analyze_tokenization(new_tokenizer, example, "Trained Tokenizer")
    
    # Compare
    if new_count < orig_count:
        print(f"\n✨ Improvement: {orig_count - new_count} fewer tokens used!")
    elif new_count == orig_count:
        print(f"\n🔄 Same number of tokens used.")
    else:
        print(f"\n⚠️ New tokenizer uses {new_count - orig_count} more tokens.")


==== Test Example 1 ====


--- Original Tokenizer Results ---
Text: བྱང་ཆུབ་ཀྱི་སེམས་རྣམ་པ་གཉིས་ཡོད་རེད།
Token count: 108
Tokens: ['à', '½', 'ĸ', 'à', '¾', '±', 'à', '½', 'Ħ', 'à', '¼', 'ĭ', 'à', '½', 'Ĩ', 'à', '½', '´', 'à', '½', 'ĸ', 'à', '¼', 'ĭ', 'à', '½', 'Ģ', 'à', '¾', '±', 'à', '½', '²', 'à', '¼', 'ĭ', 'à', '½', '¦', 'à', '½', 'º', 'à', '½', 'ĺ', 'à', '½', '¦', 'à', '¼', 'ĭ', 'à', '½', '¢', 'à', '¾', '£', 'à', '½', 'ĺ', 'à', '¼', 'ĭ', 'à', '½', 'Ķ', 'à', '¼', 'ĭ', 'à', '½', 'Ĥ', 'à', '½', 'ī', 'à', '½', '²', 'à', '½', '¦', 'à', '¼', 'ĭ', 'à', '½', '¡', 'à', '½', '¼', 'à', '½', 'ĳ', 'à', '¼', 'ĭ', 'à', '½', '¢', 'à', '½', 'º', 'à', '½', 'ĳ', 'à', '¼', 'į']
Roundtrip successful: True

--- Trained Tokenizer Results ---
Text: བྱང་ཆུབ་ཀྱི་སེམས་རྣམ་པ་གཉིས་ཡོད་རེད།
Token count: 32
Tokens: ['à½ĸ', 'à¾±', 'à½Ħ', 'à¼ĭ', 'à½Ĩ', 'à½´', 'à½ĸ', 'à¼ĭ', 'à½Ģ', 'à¾±à½²à¼ĭ', 'à½¦', 'à½º', 'à½ĺà½¦', 'à¼ĭ', 'à½¢', 'à¾£', 'à½ĺ', 'à¼ĭ', 'à½Ķ', 'à¼ĭ', 'à½Ĥà½ī', 'à½²', 'à½¦', 'à¼ĭ', 'à½¡', 'à½¼', 'à½ĳ

## 6. Analyze the New Vocabulary

Let's analyze what new tokens were added and check for Tibetan-specific tokens.

In [43]:
# Get vocabularies
old_vocab = tokenizer.get_vocab()
new_vocab = new_tokenizer.get_vocab()

# Find new tokens
new_tokens = [token for token in new_vocab.keys() if token not in old_vocab]
print(f"Added {len(new_tokens)} new tokens to the vocabulary")

# Find Tibetan tokens
tibetan_range = (0x0F00, 0x0FFF)  # Unicode range for Tibetan
new_tibetan_tokens = [token for token in new_tokens 
                      if any(ord(c) >= tibetan_range[0] and ord(c) <= tibetan_range[1] for c in token)]

print(f"Added {len(new_tibetan_tokens)} new Tibetan tokens")

# Show some examples
if new_tibetan_tokens:
    print("\nSample new Tibetan tokens:")
    for token in new_tibetan_tokens[:20]:  # Show up to 20
        print(token)

Added 2695 new tokens to the vocabulary
Added 0 new Tibetan tokens


## 7. Save the Trained Tokenizer

Save the new tokenizer for future use.

In [44]:
# Define output directory
OUTPUT_DIR = "data/whisper_latin_tibetan_tokenizer"

# Create directory if it doesn't exist
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Save the tokenizer
new_tokenizer.save_pretrained(OUTPUT_DIR)
print(f"💾 Trained tokenizer saved to: {OUTPUT_DIR}")

# Test loading it back
loaded_tokenizer = PreTrainedTokenizerFast.from_pretrained(OUTPUT_DIR)
print(f"✅ Successfully loaded tokenizer with {len(loaded_tokenizer)} tokens")

💾 Trained tokenizer saved to: data/whisper_latin_tibetan_tokenizer
✅ Successfully loaded tokenizer with 3059 tokens


## 8. Next Steps

Now that you've trained a tokenizer for Tibetan, here's how you can use it with a Whisper model:

```python
from transformers import WhisperForConditionalGeneration, PreTrainedTokenizerFast

# Load your trained tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("data/whisper_tibetan_tokenizer")

# Load a Whisper model
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Resize the token embeddings to match your tokenizer
model.resize_token_embeddings(len(tokenizer))

# Now you can fine-tune or use the model with your tokenizer
```

Remember to always resize the token embeddings of the model after loading your custom tokenizer!